# Iterative Random Shuffling Volume Denoising (IRSVD)

In [ ]:
local_debug = True

In [ ]:
try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False

if IN_COLAB:
    print("Running in Colab")
    !pip install cupy-cuda12x
    !pip install opticalflow3D
    !apt install libcudart11.0
    !apt install libcublas11
    !apt install libcufft10
    !apt install libcusparse11
    !apt install libnvrtc11.2
    from google.colab import drive
    drive.mount('/content/drive')
    !cp drive/Shareddrives/TomogramDenoising/tomograms/{vol_name}.tif .
else:
    print("Running in locahost")
    !cp ~/Downloads/{vol_name}.tif .

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

In [ ]:
import opticalflow3D
import warnings
from numba.core.errors import NumbaPerformanceWarning
import numpy as np

warnings.filterwarnings("ignore", category=NumbaPerformanceWarning)

In [ ]:
if local_debug:
    !ln -sf ../../information_theory/src/information_theory/ .
else:
    !pip install "information_theory @ git+https://github.com/vicente-gonzalez-ruiz/information_theory"
import information_theory  # pip install "information_theory @ git+https://github.com/vicente-gonzalez-ruiz/information_theory"

In [ ]:
import skimage.io

In [ ]:
import RSIVD

In [ ]:
from collections import namedtuple
Args = namedtuple("args", "input")
args = Args("epfl1_subset1_float.tif")

In [ ]:
%%bash -s "$args.input"
set -x
OUTPUT_FILENAME=$1
#rm -f $OUTPUT_FILENAME
if test ! -f $OUTPUT_FILENAME ; then
    FILEID="1iui018SGKa5nb0ybeoUAB7uCUhl9EJ5o"
    wget --no-check-certificate 'https://docs.google.com/uc?export=download&id='$FILEID -O $OUTPUT_FILENAME 2> /dev/null
fi
set +x
#https://drive.google.com/file/d/1iui018SGKa5nb0ybeoUAB7uCUhl9EJ5o/view?usp=sharing

In [ ]:
noisy = opticalflow3D.helpers.load_image(args.input)

In [ ]:
noisy = (255*(noisy - np.min(noisy))/(np.max(noisy) - np.min(noisy))).astype(np.uint8)

In [ ]:
noisy = noisy[80:, 200:, 200:]

In [ ]:
noisy.shape

In [ ]:
if IN_COLAB:
    block_size = (noisy.shape[0], noisy.shape[1], noisy.shape[2])
else:
    block_size = (50, 384, 512)
block_size = (noisy.shape[0], noisy.shape[1], noisy.shape[2])
print(block_size)

In [ ]:
"""Farneback3D class used to instantiate the algorithm with its parameters.

Args:
    iters (int): number of iterations. Defaults to 5
    num_levels (int): number of pyramid levels. Defaults to 5
    scale (float): Scaling factor used to generate the pyramid levels. Defaults to 0.5
    spatial_size (int): size of the support used in the calculation of the standard deviation of the Gaussian
        applicability. Defaults to 9.
    sigma_k (float): scaling factor used to calculate the standard deviation of the Gaussian applicability. The
        formula to calculate sigma is sigma_k*(spatial_size - 1). Defaults to 0.15.
    filter_type (str): Defines the type of filter used to average the calculated matrices. Defaults to "box"
    filter_size (int): Size of the filter used to average the matrices. Defaults to 21
    presmoothing (int): Standard deviation used to perform Gaussian smoothing of the images. Defaults to None
    device_id (int): Device id of the GPU. Defaults to 0
"""
#farneback = opticalflow3D.Farneback3D(iters=5, num_levels=3, scale=0.5, spatial_size=11, presmoothing=3, filter_type="gaussian", filter_size=11); RS_sigma = 1.0
#farneback = opticalflow3D.Farneback3D(iters=5, num_levels=2, scale=0.5, spatial_size=5, sigma_k=0.5, filter_type="gaussian", filter_size=9, presmoothing=None, device_id=0); RS_sigma = 1.0
farneback = opticalflow3D.Farneback3D(iters=5, num_levels=2, scale=0.5, spatial_size=5, sigma_k=1.0, filter_type="gaussian", filter_size=9, presmoothing=None, device_id=0); RS_sigma = 0.75
farneback = opticalflow3D.Farneback3D(iters=5, num_levels=2, scale=0.5, spatial_size=5, sigma_k=1.0, filter_type="gaussian", filter_size=17, presmoothing=None, device_id=0); RS_sigma = 1.75

In [ ]:
denoised = RSIVD.filter(farneback, block_size, noisy, RS_sigma=RS_sigma, N_iters=20)

In [ ]:
np.min(denoised)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(16, 32))
axs[0].imshow(noisy[1], cmap="gray")
axs[0].set_title(f"Noisy")
axs[1].imshow(denoised[1], cmap="gray")
axs[1].set_title(f"Denoised (DQI={information_theory.information.compute_quality_index(noisy[16], denoised[16])})")
fig.tight_layout()
plt.show()

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(16, 32))
axs[0].imshow(noisy[1][600:,800:1000], cmap="gray")
axs[0].set_title(f"Noisy")
axs[1].imshow(denoised[1][600:,800:1000], cmap="gray")
axs[1].set_title(f"Denoised (DQI={information_theory.information.compute_quality_index(noisy[16][600:,800:1000], denoised[16][600:,800:1000])}")
fig.tight_layout()
plt.show()

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(16, 32))
axs[0].imshow(noisy[:, 100], cmap="gray")
axs[0].set_title(f"Noisy")
axs[1].imshow(denoised[:, 100], cmap="gray")
axs[1].set_title(f"Denoised (DQI={information_theory.information.compute_quality_index(noisy[:, 100], denoised[:, 100])})")
fig.tight_layout()
plt.show()

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(16, 32))
axs[0].imshow(noisy[:, :, 100], cmap="gray")
axs[0].set_title(f"Noisy")
axs[1].imshow(denoised[:, :, 100], cmap="gray")
axs[1].set_title(f"Denoised (DQI={information_theory.information.compute_quality_index(noisy[:, :, 100], denoised[:, :, 100])})")
fig.tight_layout()
plt.show()

In [ ]:
figure(figsize=(32, 32))
plt.subplot(1, 3, 1)
plt.title("original")
imgplot = plt.imshow(noisy[5][::-1, :], cmap="gray")
plt.subplot(1, 3, 2)
plt.title("$\sigma_\mathrm{RS}=$"+f"{RS_sigma}")
plt.imshow(denoised[5][::-1, :], cmap="gray")
plt.subplot(1, 3, 3)
plt.title("difference")
plt.imshow(noisy[5][::-1, :] - denoised[5][::-1, :], cmap="gray")

In [ ]:
input()

In [ ]:
farneback = opticalflow3D.Farneback3D(iters=5,
                                      num_levels=3,
                                      scale=0.5,
                                      spatial_size=5,
                                      presmoothing=4,
                                      filter_type="box",
                                      filter_size=5,
                                     )

In [ ]:
RS_sigma = 1.0
denoised_vol = RSIVD.filter(farneback, block_size, noisy_vol, RS_sigma=RS_sigma, N_iters=25)

In [ ]:
figure(figsize=(32, 32))
plt.subplot(1, 3, 1)
plt.title("original")
imgplot = plt.imshow(noisy_vol[75][::-1, :], cmap="gray")
plt.subplot(1, 3, 2)
plt.title("$\sigma_\mathrm{RS}=$"+f"{RS_sigma}")
plt.imshow(denoised_vol[75][::-1, :], cmap="gray")
plt.subplot(1, 3, 3)
plt.title("difference")
plt.imshow(noisy_vol[75][::-1, :] - denoised_vol[75][::-1, :], cmap="gray")

In [ ]:
skimage.io.imsave(f"{vol_name}_denoised_{RS_sigma}.tif", denoised_vol, imagej=True)